# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## Question 1

To answer if there is a difference between the callback rates, we would like to do a difference of proportions test. To check if the conditions of the test apply, we count the number of samples of each type we have. 

In [11]:
call = data.call
call_b = data.call[data.race == "b"]
call_w = data.call[data.race == "w"]
np.array([[np.sum(call_b), np.sum(call_w)],[len(call_b), len(call_w)]])

array([[ 157.,  235.],
       [2435., 2435.]])

As we can see, since $np > 10$ and $n(1-p) > 10$ we can say that the CLT applies and we can proceed with the test. 

## Question 2

For our difference in proportion test, we have $H_0: p_b = p_w$ and $H_a: p_b \neq p_w$, where $p_b$ is the proportion of black named people getting callbacks aand $p_w$ is the proportion of white named people getting callbacks. 

## Question 3

We first do the bootstrap approach:

In [16]:
p_b = np.sum(call_b) / len(call_b)
p_w = np.sum(call_w) / len(call_w)
p_diff = p_b - p_w
p_diff

-0.032032854209445585

In [18]:
def draw_bs_replica_paired(data, func, n, size=1):
    # This function draws the replicates
    replicates_a = np.empty(size)
    replicates_b = np.empty(size)
    for i in range(size):
        replicates = np.random.choice(data, size=len(data))
        replicates_a[i] = func(replicates[:n])
        replicates_b[i] = func(replicates[n:])
    return replicates_a, replicates_b
np.random.seed(3)
rep_B_tot, rep_W_tot = draw_bs_replica_paired(call, np.sum, len(call_b), 100000)
rep_B_p = rep_B_tot / len(call_b)
rep_W_p = rep_W_tot / len(call_w)
pval_boot = np.sum(rep_B_p - rep_W_p <= p_diff) / 100000
pval_boot

2e-05

This shows that the p-value is extremely small. We shall now construct a confidence interval of the difference in proportion:

In [26]:
se = np.sqrt((p_b*(1-p_b))/len(call_b) + (p_w*(1-p_w))/len(call_w))
margin = 1.96 * se
(p_diff - margin, p_diff + margin)

(-0.047288260559332024, -0.016777447859559147)

As we can see, the confidence interval does not contain 0, so there is sufficient evidence to reject the null hypothesis. 

## Question 4

Our testing results show that there is a significant disadvantage to getting callbacks by having a black-sounding name versus a white-sounding one. We believe that this supports the idea that racial discrimination continues to be pervasive among employers in the United States labour market.

## Question 5

This test is insufficient to determine if perceived race given the name is the most important factor in callback success. While the test results show that the value is significant, it could be because there is some underlying cause which is correlated to race which is the issue (e.g. it could be high socioeconomic standards being the important factor in callbac success, and high socioeconomic standards is correlated with a white name in the US). To be able to examine the issue of most important factor, we would need to consider each factor at the same time, using perhaps a random forest or Boruta feature selection to be able to measure the efficacy of the factors. 